# Macros

Macros in SAS write a program that writes a program. When you run SAS code (i.e. any PROC), SAS compiles and immediately executes it. With macros, SAS pass macro statements to the macro processor which results in SAS code.

## Macros vs. Macro variables

* Macro variable (&) - Can contanin any value
* Macro (%) - A program that contains DATA and PROC steps, and macro statements.

Macro variables can be eihter local or global. It is local when the variable is defined inside a macro.

## Some examples

### %LET macro

%LET is a macro statement that assigns a value to a macro variable. It has the next form:

`%LET macro-variable-name = value;`

The next dataset represents an inventory of a company that manufactures bicycles. For each model they record its name, class (Road, Track, or Mountain), list price, and frame material.

What if we want to subset by class? Let's do it without using macros.


In [46]:
DATA models;
    infile DATALINES truncover;
    input Model $ 1-12  Class $ Price Frame$ 28-38;
    DATALINES;
Black Bora   Track     796 Aluminum
Delta Breeze Road      399 CroMoly
Jet Stream   Track    1130 CroMoly
Mistral      Road     1995 Carbon Comp
Nor'easter   Mountain  899 Aluminum
Santa Ana    Mountain  459 Aluminum
Scirocco     Mountain 2256 Titanium
Trade Wind   Road      759 Aluminum
;
RUN;

PROC PRINT DATA = models NOOBS;
    where Class = "Mountain";
    format Price dollar6.;
    title "Current Models of mountain Bicycles";
RUN;

Model,Class,Price,Frame
Nor'easter,Mountain,$899,Aluminum
Santa Ana,Mountain,$459,Aluminum
Scirocco,Mountain,"$2,256",Titanium


What if we want to subset by other class? We can now start using the `%LET` macro

In [47]:
 %LET bikeclass = Road;
 
 PROC PRINT DATA = models NOOBS;
 WHERE Class = "&bikeclass";
 FORMAT Price DOLLAR6.;
TITLE "Current Models of &bikeclass Bicycles";
 RUN;
 

Model,Class,Price,Frame
Delta Breeze,Road,$399,CroMoly
Mistral,Road,"$1,995",Carbon Comp
Trade Wind,Road,$759,Aluminum


### Own macros

ALl macros start with `%MACRO`and end with `%MEND`. Inside these two statements you can write your own SAS code. Let's see an simple example of this:

In [49]:
%MACRO printit;
PROC PRINT data = models NOOBS;
    title 'Current Models';
    var Model Class Frame Price;
    format Price dollar6.;
RUN;
%MEND printit;

In [50]:
%printit

Model,Class,Frame,Price
Black Bora,Track,Aluminum,$796
Delta Breeze,Road,CroMoly,$399
Jet Stream,Track,CroMoly,"$1,130"
Mistral,Road,Carbon Comp,"$1,995"
Nor'easter,Mountain,Aluminum,$899
Santa Ana,Mountain,Aluminum,$459
Scirocco,Mountain,Titanium,"$2,256"
Trade Wind,Road,Aluminum,$759


#### Adding parameters

Sometimes called arguments (programming languages), parameters are macro variables that are set whem invoking a macro. 

*Question*

1. Are parameters local or global macro variables?

Using the previous example, we can add 2 parameters that will sort our data by a variable from the dataset.



In [65]:

 %MACRO sortandprint(sortseq=, sortvar=);
 PROC SORT DATA = models; BY &sortseq &sortvar;
 PROC PRINT DATA = models NOOBS;
 TITLE 'Current Models';
 TITLE2 "Sorted by &sortseq &sortvar";
 VAR Model Class Frame Price;
 FORMAT Price DOLLAR6.;
RUN;
%MEND sortandprint;
%sortandprint(sortseq=, sortvar=Class)

Model,Class,Frame,Price
Nor'easter,Mountain,Aluminum,$899
Santa Ana,Mountain,Aluminum,$459
Scirocco,Mountain,Titanium,"$2,256"
Delta Breeze,Road,CroMoly,$399
Mistral,Road,Carbon Comp,"$1,995"
Trade Wind,Road,Aluminum,$759
Black Bora,Track,Aluminum,$796
Jet Stream,Track,CroMoly,"$1,130"


*Question*

2. How can we sort by descending values?

#### Conditional statements

We can also include conditional statements in our macros. We will use statements such as `%IF`, `%THEN`, `%ELSE`, `%DO`. These statements can only be used inside a macro, and can perform actions that would be completely impossible for standard SAS statements.

Suppose that we have a number of binary dependent variables and two independent variables. Our task is to fit a logistic model for each of the dependent variables on the same two independent variables. We can do this many times using `%PROC LOGISTIC` although it can become tedious at some point. 

In [1]:
DATA temp;
  input v1-v5 ind1 ind2;
  cards;
1 0 1 1 0 34 23
0 0 1 0 1 22 32
1 1 1 0 0 12 10
0 1 0 1 1 56 90
0 1 0 1 1 26 80
1 1 0 0 0 46 45
0 0 0 1 1 57 53
1 1 0 0 0 22 77
0 1 0 1 1 44 45
1 1 0 0 0 41 72
;
run;

SAS Connection established. Subprocess id is 4230



Let's run it the way we know for `v1`.

In [2]:
PROC LOGISTIC data = temp descending;
  model v1 = ind1 ind2;
RUN;

Now we can create a macro to automate this process.

In [3]:
%MACRO mylogit(num);
   %DO i = 1 %TO &num;
      title "dependent variable is v&i";
      PROC logistic data=temp des;
        model v&i = ind1 ind2;
      RUN;
  %END;
%MEND;
%mylogit(5)

## Task

Create a macro that generates a poisson random variable `randpoi` and has 3 parameters: `seed`, `lambda` and sample size `n`. In addition, include a `PROC UNIVARIATE` which prints the default plots.